In [1]:
%pwd

'd:\\Projects\\kidney-disease-classification\\research'

In [2]:
import os
os.chdir("../")
%pwd

'd:\\Projects\\kidney-disease-classification'

In [3]:
import dagshub
dagshub.init(repo_owner='bilalsadiq03', repo_name='kidney-disease-classification', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

Accessing as bilalsadiq03

Initialized MLflow to track repo "bilalsadiq03/kidney-disease-classification"

Repository bilalsadiq03/kidney-disease-classification initialized!

In [4]:
import tensorflow as tf

In [5]:
model = tf.keras.models.load_model('artifacts/training/model.h5')

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    path_of_model: Path
    tarining_data: Path
    all_params: dict
    mlflow_uri: str
    params_batch_size: int
    params_image_size: list

In [7]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [8]:
class configurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        eval_config = ModelEvaluationConfig(
            path_of_model=Path("artifacts/training/model.h5"),
            tarining_data=Path("artifacts/data_ingestion/kidney-ct-scan-image"),
            all_params=self.params,
            mlflow_uri="https://dagshub.com/bilalsadiq03/kidney-disease-classification.mlflow",
            params_batch_size=self.params.BATCH_SIZE,
            params_image_size=self.params.IMAGE_SIZE
        )
        return eval_config

In [9]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [10]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def _valid_generator(self):

        datagen_kwargs = dict(
            rescale=1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)

        self.valid_generator = valid_datagen.flow_from_directory(
            directory=self.config.tarining_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(model_path: Path) -> tf.keras.Model:
        model = tf.keras.models.load_model(model_path)
        return model
    
    def evaluate_model(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.scores = model.evaluate(self.valid_generator)
    
        

    def save_scores(self):
        scores = {"loss": self.scores[0],
                  "accuracy": self.scores[1]
        }
        save_json(path=Path("scores.json"), data=scores)
    
    def log_into_mlflow(self):
        mlflow.set_tracking_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(self.config.mlflow_uri).scheme
        

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("loss", self.scores[0])
            mlflow.log_metric("accuracy", self.scores[1])

            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")


            

In [11]:
try:
    config = configurationManager()
    eval_config = config.get_model_evaluation_config()
    evaluation= ModelEvaluation(config=eval_config)
    evaluation.evaluate_model()
    evaluation.save_scores()
    evaluation.log_into_mlflow()

except Exception as e:
    raise e

[2025-12-08 22:43:44,490: INFO: common: yaml file: configs\config.yaml loaded successfully]
[2025-12-08 22:43:44,494: INFO: common: yaml file: params.yaml loaded successfully]
[2025-12-08 22:43:44,495: INFO: common: Directory created at: artifacts]
Found 139 images belonging to 2 classes.
9/9 [==============================] - 12s 1s/step - loss: 6.6256e-05 - accuracy: 1.0000
[2025-12-08 22:43:56,971: INFO: common: json file saved at: scores.json]


2025/12/08 22:43:58 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2025-12-08 22:43:59,230: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\hp\AppData\Local\Temp\tmpua3ea220\model\data\model\assets
[2025-12-08 22:43:59,799: INFO: builder_impl: Assets written to: C:\Users\hp\AppData\Local\Temp\tmpua3ea220\model\data\model\assets]


d:\Projects\kidney-disease-classification\venv\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2025/12/08 22:45:25 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 3
Created version '3' of model 'VGG16Model'.
